In [ ]:
%pip install nilearn nibabel pandas matplotlib
%pip install --upgrade nilearn

import os
import numpy as np
import pandas as pd
import nilearn
import glob
import re
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn.image import load_img
from nilearn.plotting import plot_epi, plot_design_matrix, plot_contrast_matrix, plot_stat_map
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.contrasts import compute_contrast
from nilearn.glm.second_level import SecondLevelModel
from nilearn.plotting import plot_stat_map
from nilearn.reporting import get_clusters_table
from nilearn.image import coord_transform
from nilearn.image import resample_to_img, load_img
from nilearn.input_data import NiftiLabelsMasker



In [ ]:
print(nilearn.__version__)

In [ ]:
# 1. Paths
func_dir = '/Users/onjolikrywiak/Desktop/BrainHack/sub-001/func'
reg_dir = '/Users/onjolikrywiak/Desktop/BrainHack/events/sub-001/regressors'

In [ ]:
# 2. Grab only the Lin2009cAsym bold files
pattern   = os.path.join(func_dir, '*MNI152NLin2009cAsym*_bold.nii*')
nii_files = sorted(glob.glob(pattern))
print("Found bold files:", nii_files)

In [ ]:
# 3. Loop & pair each with its confounds
for nii_path in nii_files:
    basename = os.path.basename(nii_path)
    
    # Extract subj_task and run number, e.g. "sub-001_task-empathy" + "run-01"
    m = re.match(r'(sub-[^_]+_task-[^_]+).*?(run-\d+)', basename)
    if not m:
        raise ValueError(f"Can't parse subject/task/run from {basename}")
    subj_task = m.group(1)
    run       = m.group(2)
    
    # Build confounds filename: sub-001_task-empathy_run-01_confounds.txt
    conf_name = f"{subj_task}_{run}_confounds.txt"
    conf_path = os.path.join(reg_dir, conf_name)
    if not os.path.exists(conf_path):
        raise FileNotFoundError(f"Confounds file not found: {conf_path}")
    
    # Load the bold image
    img  = nib.load(nii_path)
    data = img.get_fdata()
    print(f"{basename} → image shape {data.shape}")
    
    # Load the confounds
    # adjust sep if whitespace-delimited; here we assume tab-delimited
    conf_df = pd.read_csv(conf_path, sep='\t', header=None)
    print(f"{conf_name} → confounds shape {conf_df.shape}")
    
    # Assign to run-specific variables
    if run == 'run-01':
        img1, data1, path1, conf1 = img, data, nii_path, conf_df
    elif run == 'run-02':
        img2, data2, path2, conf2 = img, data, nii_path, conf_df

### Trim 3 timepoints from the beginning and end of BOLD

In [ ]:
# Trim 3 timepoints from the beginning and end of BOLD
trim = 3
if data.shape[-1] > 2 * trim:
    data = data[..., trim:-trim]
    print(f"Trimmed BOLD shape: {data.shape}")
else:
    raise ValueError("Not enough timepoints to trim 3 from each end.")

### Check to see if BOLD and confounds are the same length

In [ ]:
print(f"BOLD timepoints: {data.shape[-1]}")
print(f"Confounds timepoints: {conf_df.shape[0]}")

if data.shape[-1] == conf_df.shape[0]:
    print("BOLD and confounds are aligned!")
else:
    print("Warning: BOLD and confounds are NOT aligned!")

## Generate a Z-Map for One Trial Type (Single Subject)

In [ ]:
# 1. Parse your custom events file
events_txt = '/Users/onjolikrywiak/Desktop/BrainHack/events/sub-001/regressors/sub-001_task-empathy_timing-selfpain.txt'
with open(events_txt) as f:
    line = f.readline().strip()
pairs = line.split('\t')

# Build events DataFrame
onsets = []
durations = []
for pair in pairs:
    onset, duration = pair.split(':')
    onsets.append(float(onset))
    durations.append(float(duration))
events = pd.DataFrame({
    'onset': onsets,
    'duration': durations,
    'trial_type': 'selfpain'  # or another label if needed
})

# 2. Fit the GLM (no confounds)
fmri_glm = FirstLevelModel(
    t_r=2.0,  # set your TR
    noise_model='ar1',
    standardize=True,
    hrf_model='spm',
    drift_model='cosine'
)
fmri_glm = fmri_glm.fit(img, events=events)

# 3. Inspect design matrix columns
print("Design matrix columns:", fmri_glm.design_matrices_[0].columns.tolist())

# 4. Define and compute the contrast
contrast_name = 'selfpain'
if contrast_name in fmri_glm.design_matrices_[0].columns:
    contrast_vec = (fmri_glm.design_matrices_[0].columns == contrast_name).astype(int)
    z_map = fmri_glm.compute_contrast(contrast_vec, output_type='z_score')

    # 5. Plot the result
    plot_stat_map(z_map, title=f'{contrast_name} > baseline', threshold=3.1)
else:
    print(f"Contrast '{contrast_name}' not found in design matrix columns.")

## Generate Z-Maps for All Trial Types (Single Subject)

In [ ]:
# Path to your regressors folder
reg_dir = '/Users/onjolikrywiak/Desktop/BrainHack/events/sub-001/regressors'

# Find all timing files in the folder
event_files = glob.glob(os.path.join(reg_dir, 'sub-001_task-empathy_timing-*.txt'))

for events_txt in event_files:
    # Extract trial type from filename
    basename = os.path.basename(events_txt)
    trial_type = basename.split('timing-')[1].replace('.txt', '')

    # Parse the custom events file
    with open(events_txt) as f:
        line = f.readline().strip()
    pairs = line.split('\t')

    # Build events DataFrame
    onsets = []
    durations = []
    for pair in pairs:
        onset, duration = pair.split(':')
        onsets.append(float(onset))
        durations.append(float(duration))
    events = pd.DataFrame({
        'onset': onsets,
        'duration': durations,
        'trial_type': trial_type
    })

    # Fit the GLM (no confounds)
    fmri_glm = FirstLevelModel(
        t_r=2.0,
        noise_model='ar1',
        standardize=True,
        hrf_model='spm',
        drift_model='cosine'
    )
    fmri_glm = fmri_glm.fit(img, events=events)

    # Inspect design matrix columns
    print(f"Design matrix columns for {trial_type}:", fmri_glm.design_matrices_[0].columns.tolist())

    # Define and compute the contrast
    contrast_name = trial_type
    if contrast_name in fmri_glm.design_matrices_[0].columns:
        contrast_vec = (fmri_glm.design_matrices_[0].columns == contrast_name).astype(int)
        z_map = fmri_glm.compute_contrast(contrast_vec, output_type='z_score')

        # Plot the result
        plot_stat_map(z_map, title=f'{contrast_name} > baseline', threshold=3.1)
    else:
        print(f"Contrast '{contrast_name}' not found in design matrix columns.")

## Generate Z-Maps for All Trial Types (All Subjects)

In [ ]:
base_dir = '/Users/onjolikrywiak/Desktop/BrainHack'
output_dir = os.path.join(base_dir, 'outputs')
os.makedirs(output_dir, exist_ok=True)

subjects = sorted([d for d in os.listdir(base_dir) if d.startswith('sub-')])[36:]
print("Subjects found:", subjects)

for subj in subjects:
    print("Processing subject:", subj)
    func_dir = os.path.join(base_dir, subj, 'func')
    reg_dir = os.path.join(base_dir, 'events', subj, 'regressors')
    nii_files = sorted(glob.glob(os.path.join(func_dir, '*MNI152NLin2009cAsym*_bold.nii*')))
    print(f"Subject: {subj}, nii_files: {nii_files}")
    
    for nii_path in nii_files:
        img = nib.load(nii_path)
        basename = os.path.basename(nii_path)
        subj_task_match = re.match(r'(sub-[^_]+_task-[^_]+)', basename)
        if not subj_task_match:
            print(f"Could not parse subj_task from {basename}")
            continue
        subj_task = subj_task_match.group(1)
        
        event_files = glob.glob(os.path.join(reg_dir, f'{subj_task}_timing-*.txt'))
        print(f"Subject: {subj}, event_files: {event_files}")
        for events_txt in event_files:
            print(f"Processing event file: {events_txt}")
            trial_type = os.path.basename(events_txt).split('timing-')[1].replace('.txt', '')
            with open(events_txt) as f:
                line = f.readline().strip()
            pairs = line.split('\t')
            onsets, durations = [], []
            for pair in pairs:
                onset, duration = pair.split(':')
                onsets.append(float(onset))
                durations.append(float(duration))
            events = pd.DataFrame({
                'onset': onsets,
                'duration': durations,
                'trial_type': trial_type
            })
            
            fmri_glm = FirstLevelModel(
                t_r=2.0,
                noise_model='ar1',
                standardize=True,
                hrf_model='spm',
                drift_model='cosine'
            )
            fmri_glm = fmri_glm.fit(img, events=events)
            print(f"Subject: {subj}, Trial: {trial_type}")
            print("Design matrix columns:", fmri_glm.design_matrices_[0].columns.tolist())
            
            contrast_name = trial_type
            if contrast_name in fmri_glm.design_matrices_[0].columns:
                contrast_vec = (fmri_glm.design_matrices_[0].columns == contrast_name).astype(int)
                z_map = fmri_glm.compute_contrast(contrast_vec, output_type='z_score')
                print(f"About to plot and save for {subj} {trial_type}")
                zdata = z_map.get_fdata()
                print("z_map min:", np.nanmin(zdata), "max:", np.nanmax(zdata), "mean:", np.nanmean(zdata))
                
                # Save z-map NIfTI
                zmap_nii = os.path.join(output_dir, f"{subj}_{trial_type}_zmap.nii.gz")
                z_map.to_filename(zmap_nii)
                
                # Save plot as PNG
                plot_png = os.path.join(output_dir, f"{subj}_{trial_type}_zmap.png")
                disp = plot_stat_map(
                    z_map,
                    title=f'{subj} {trial_type} > baseline',
                    threshold=3.1,
                    output_file=plot_png
                )
                plt.close()  # Close the figure to avoid displaying in notebook
                print(f"Saved: {zmap_nii} and {plot_png}")
            else:
                print(f"Contrast '{contrast_name}' not found in design matrix columns for {subj}.")

# Second-level (group) analysis

### Download/load atlas

In [ ]:
from nilearn.datasets import fetch_atlas_harvard_oxford
# Fetch the Harvard-Oxford atlas
atlas = fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_img = load_img(atlas.maps)
atlas_labels = list(atlas.labels)

### Run group level analysis with threshold and atlas

In [ ]:
trial_types = ["otherneutralanticipation",
               "otherneutralcue",
               "othernopain",
               "othernopainrest",
               "otherpain",
               "otherpainanticipation",
               "otherpaincue",
               "otherpainrest",
               "selfneutralanticipation",
               "selfneutralcue",
               "selfnopain",
               "selfnopainrest",
               "selfpain",
               "selfpainanticipation",
               "selfpainrest"]

In [ ]:
for trial_type in trial_types:
    zmap_files = sorted(glob.glob(f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/sub-*_{trial_type}_zmap.nii.gz'))
    if not zmap_files:
        print(f"No zmaps found for {trial_type}, skipping.")
        continue
    zmap_imgs = [load_img(f) for f in zmap_files]
    print(f"Found zmaps for {trial_type}:", zmap_files)

    # Second-level analysis
    n_subs = len(zmap_imgs)
    design_matrix = pd.DataFrame([1] * n_subs, columns=['intercept'])
    second_level_model = SecondLevelModel()
    second_level_model = second_level_model.fit(zmap_imgs, design_matrix=design_matrix)
    zmap_group = second_level_model.compute_contrast('intercept', output_type='z_score')

    from nilearn.glm import threshold_stats_img
    zmap_group = second_level_model.compute_contrast('intercept', output_type='z_score')
    
    # FDR correction ## Activation is too weak to survive FDR or FWE correction
    #thresholded_map, threshold = threshold_stats_img(
    #    zmap_group, alpha=0.1, height_control='fdr'
    #    )
    #print(f"FDR threshold used: {threshold}")

    # Plot and save
    plot_stat_map(
        zmap_group,
        threshold=3.1,
        title=f'Group-level {trial_type} z-map',
        output_file=f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_zmap.png'
    )
    zmap_group.to_filename(f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_zmap.nii.gz')
    print(f"Saved group z-map for {trial_type}")

    # Get and save clusters table
    table = get_clusters_table(zmap_group, stat_threshold=3.1)
    
    # Resample atlas to match your group z-map
    resampled_atlas = resample_to_img(atlas_img, zmap_group, interpolation='nearest')
    
    def get_label_from_coords(coord):
        from nilearn.image import coord_transform
        import numpy as np
        # Convert MNI coordinates to voxel indices
        voxel_coords = np.round(coord_transform(*coord, np.linalg.inv(resampled_atlas.affine))).astype(int)
        data = resampled_atlas.get_fdata()
        try:
            label_index = int(data[tuple(voxel_coords)])
            if label_index > 0 and label_index < len(atlas_labels):
                return atlas_labels[label_index]
            else:
                return 'Unknown'
        except Exception:
            return 'Unknown'
        
    # Add anatomical labels to the cluster table
    if not table.empty and all(col in table.columns for col in ['X', 'Y', 'Z']):
        table['peak_coord'] = list(zip(table['X'], table['Y'], table['Z']))
        table['Region_Label'] = table['peak_coord'].apply(get_label_from_coords)
    else:
        table['peak_coord'] = [None] * len(table)
        table['Region_Label'] = [None] * len(table)

    # Save table to CSV
    csv_path = f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_clusters.csv'
    table.to_csv(csv_path, index=False)
    print(f"Saved clusters table for {trial_type} to {csv_path}")

In [ ]:
for trial_type in trial_types:
    zmap_files = sorted(glob.glob(f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/sub-*_{trial_type}_zmap.nii.gz'))
    if not zmap_files:
        print(f"No zmaps found for {trial_type}, skipping.")
        continue
    zmap_imgs = [load_img(f) for f in zmap_files]
    print(f"Found zmaps for {trial_type}:", zmap_files)

    # Second-level analysis
    n_subs = len(zmap_imgs)
    design_matrix = pd.DataFrame([1] * n_subs, columns=['intercept'])
    second_level_model = SecondLevelModel()
    second_level_model = second_level_model.fit(zmap_imgs, design_matrix=design_matrix)
    zmap_group = second_level_model.compute_contrast('intercept', output_type='z_score')

    from nilearn.glm import threshold_stats_img
    zmap_group = second_level_model.compute_contrast('intercept', output_type='z_score')
    
    # FDR correction ## Activation is too weak to survive FDR or FWE correction
    #thresholded_map, threshold = threshold_stats_img(
    #    zmap_group, alpha=0.1, height_control='fdr'
    #    )
    #print(f"FDR threshold used: {threshold}")

    # Plot and save
    plot_stat_map(
        zmap_group,
        threshold=3.1,
        title=f'Group-level {trial_type} z-map',
        output_file=f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_zmap.png'
    )
    zmap_group.to_filename(f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_zmap.nii.gz')
    print(f"Saved group z-map for {trial_type}")

    # Get and save clusters table
    table = get_clusters_table(zmap_group, stat_threshold=3.1)
    
    # Resample atlas to match your group z-map
    resampled_atlas = resample_to_img(atlas_img, zmap_group, interpolation='nearest')
    
    def get_label_from_coords(coord):
        from nilearn.image import coord_transform
        import numpy as np
        # Convert MNI coordinates to voxel indices
        voxel_coords = np.round(coord_transform(*coord, np.linalg.inv(resampled_atlas.affine))).astype(int)
        data = resampled_atlas.get_fdata()
        try:
            label_index = int(data[tuple(voxel_coords)])
            if label_index > 0 and label_index < len(atlas_labels):
                return atlas_labels[label_index]
            else:
                return 'Unknown'
        except Exception:
            return 'Unknown'
        
    # Add anatomical labels to the cluster table
    if not table.empty and all(col in table.columns for col in ['X', 'Y', 'Z']):
        table['peak_coord'] = list(zip(table['X'], table['Y'], table['Z']))
        table['Region_Label'] = table['peak_coord'].apply(get_label_from_coords)
    else:
        table['peak_coord'] = [None] * len(table)
        table['Region_Label'] = [None] * len(table)

    # Save table to CSV
    csv_path = f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_clusters.csv'
    table.to_csv(csv_path, index=False)
    print(f"Saved clusters table for {trial_type} to {csv_path}")

In [20]:
import pandas as pd
import glob
from collections import Counter

summary = {}

# Loop through all cluster tables
for trial_type in trial_types:
    csv_path = f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_clusters.csv'
    try:
        table = pd.read_csv(csv_path)
        # Count occurrences of each region label
        region_counts = Counter(table['Region_Label'].dropna())
        summary[trial_type] = dict(region_counts)
    except Exception as e:
        print(f"Could not process {csv_path}: {e}")

# Print summary for each trial type
for trial_type, regions in summary.items():
    print(f"\n{trial_type}:")
    for region, count in regions.items():
        print(f"  {region}: {count} clusters")

# Optionally, save the summary to a CSV
summary_df = pd.DataFrame(summary).fillna(0).astype(int)
summary_df.to_csv('/Users/onjolikrywiak/Desktop/BrainHack/outputs/region_summary.csv')
print("Saved region summary to outputs/region_summary.csv")

Could not process /Users/onjolikrywiak/Desktop/BrainHack/outputs/group_otherpaincue_clusters.csv: [Errno 2] No such file or directory: '/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_otherpaincue_clusters.csv'

otherneutralanticipation:
  Superior Temporal Gyrus, posterior division: 1 clusters
  Planum Temporale: 3 clusters
  Unknown: 14 clusters
  Superior Parietal Lobule: 1 clusters
  Middle Frontal Gyrus: 3 clusters
  Superior Frontal Gyrus: 1 clusters
  Precentral Gyrus: 1 clusters
  Parietal Opercular Cortex: 1 clusters
  Postcentral Gyrus: 1 clusters
  Temporal Pole: 1 clusters
  Supramarginal Gyrus, posterior division: 2 clusters
  Inferior Temporal Gyrus, temporooccipital part: 1 clusters
  Parahippocampal Gyrus, posterior division: 1 clusters

otherneutralcue:
  Superior Parietal Lobule: 1 clusters
  Unknown: 7 clusters
  Lateral Occipital Cortex, inferior division: 1 clusters
  Frontal Opercular Cortex: 1 clusters
  Temporal Pole: 1 clusters
  Middle Frontal Gyrus: 2 cl